In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import confusion_matrix  

In [4]:
reduced = pd.read_csv("reduced.csv", index_col="date")
d = reduced.apply(np.log).diff()[1:]
data = []
for podatak in d["ABBV"]:
    data.append(podatak)
d.shape
print(reduced)
print(d)

                 ABBV        ABC        ABT         ACN    ADBE        ADI  \
date                                                                         
2013-06-03  37.035571  49.114233  33.653966   74.327146   43.46  40.921428   
2013-06-04  37.266189  49.654974  33.327932   73.557620   44.39  41.198287   
2013-06-05  36.762246  49.319342  32.875106   72.045728   43.84  40.537399   
2013-06-06  37.155151  48.993032  33.183027   72.018569   43.57  40.608847   
2013-06-07  37.582221  49.356634  33.970943   73.240757   44.12  41.269734   
...               ...        ...        ...         ...     ...        ...   
2017-03-31  63.502713  87.307568  43.689744  117.506015  130.13  81.046642   
2017-04-03  63.376020  86.153332  43.748771  115.310374  129.59  80.344469   
2017-04-04  63.453985  85.758721  43.463474  115.702453  130.04  80.344469   
2017-04-05  63.307800  86.005353  43.089637  115.310374  129.89  79.157697   
2017-04-06  63.424748  86.459155  42.961746  114.496811  130.15 

In [6]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2, L - l - s)]).astype(np.float32)
print(len(X[0]))
print(X)
print(Y_test)

20
[[ 0.00620763 -0.01361507  0.01063102 ... -0.00577969 -0.04238826
   0.00650997]
 [-0.01361507  0.01063102  0.0114287  ... -0.04238826  0.00650997
   0.01431664]
 [ 0.01063102  0.0114287  -0.00181984 ...  0.00650997  0.01431664
   0.00047371]
 ...
 [-0.02490616 -0.00157743  0.01046947 ... -0.01601987  0.02235665
  -0.01301865]
 [-0.00157743  0.01046947  0.01396213 ...  0.02235665 -0.01301865
  -0.01489014]
 [ 0.01046947  0.01396213  0.00665361 ... -0.01301865 -0.01489014
  -0.00527215]]
[1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1.
 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 

In [4]:
len(X), len(X_test), Y.mean(), Y_test.mean()

(463, 464, 0.49892008, 0.48922414)

In [6]:
n = len(X)
m = len(X_test)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

In [7]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X[i], X[j], radius = 5, dist = 2)[0]


for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X[j], radius = 5, dist = 2)[0]
                      
print(time.time()-t0)

4251.3177053928375


In [7]:
kernel_train = np.load("kernel_train_univariant_ABBV.npy")

In [8]:
kernel_test = np.load("kernel_test_univariant_ABBV.npy")

In [10]:
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
clf.fit(kernel, Y)
#res = clf.predict(testkernel) #test
res = clf.predict(kernel) #train
#print('accuracy test: ', np.mean(res == Y_test))
#print('AUC test:', roc_auc_score(Y_test, res))
print('accuracy train: ', np.mean(res == Y))
print('AUC train:', roc_auc_score(Y, res))

accuracy train:  0.7580993520518359
AUC train: 0.7580515748619197


In [8]:
cm = confusion_matrix(Y_test, res)
cm

array([[166,  71],
       [ 54, 173]], dtype=int64)